### Exercises

Using the repo setup directions, setup a new local and remote repository named spark-exercises. The local version of your repo should live inside of ~/codeup-data-science. This repo should be named spark-exercises

Save this work in your spark-exercises repo. Then add, commit, and push your changes.

Create a jupyter notebook or python script named spark101 for this exercise.

In [91]:
import pyspark
import pandas as pd
import numpy as np
from pyspark.sql.functions import lit, round, col
from pyspark.sql.functions import regexp_extract, regexp_replace

In [65]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()


In [3]:
spark

 1. Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [16]:
#create panda dataframe
df = pd.DataFrame({"language":["python","sql","ruby","c","java","c++"]})
df

,language
0,python
1,sql
2,ruby
3,c
4,java
5,c++


In [17]:
#now create spark dataframe 
spark_df= spark.createDataFrame(df)
spark_df.show()

+--------+
|language|
+--------+
|  python|
|     sql|
|    ruby|
|       c|
|    java|
|     c++|
+--------+



In [18]:
#view schema
spark_df.printSchema()

root
 |-- language: string (nullable = true)



In [19]:
#shape of dataframe, not as simple as pandas
#import count
from pyspark.sql.functions import count, avg
spark_df.count(), len(spark_df.columns)

(6, 1)

In [8]:
#show first 5 records
spark_df.show(5)

+--------+
|language|
+--------+
|  python|
|     sql|
|    ruby|
|       c|
|    java|
+--------+
only showing top 5 rows



### 2.Load the mpg dataset as a spark dataframe.

In [6]:
#import data where mpg resides
from pydataset import data

In [10]:
#get mpg pyspark data
mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)                         

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



#### a. Create 1 column of output that contains a message like the one below:
    The 1999 audi a4 has a 4 cylinder engine

In [11]:
from pyspark.sql.functions import concat
mpg.select(concat(lit("The "), "year", lit(" "), "manufacturer",lit(" "), "model", lit(" has a "), "cyl", lit(" cylinder engine")).alias("concat")).show(5,truncate = False)

+----------------------------------------+
|concat                                  |
+----------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine|
|The 1999 audi a4 has a 4 cylinder engine|
|The 2008 audi a4 has a 4 cylinder engine|
|The 2008 audi a4 has a 4 cylinder engine|
|The 1999 audi a4 has a 6 cylinder engine|
+----------------------------------------+
only showing top 5 rows



#### b. Transform the trans column so that it only contains either manual or auto.


In [12]:
mpg.select("trans").show(1)

+--------+
|   trans|
+--------+
|auto(l5)|
+--------+
only showing top 1 row



In [13]:
mpg.select("trans",regexp_extract("trans" , r"(\w+)", 1).alias("transmission")).show(4)

+----------+------------+
|     trans|transmission|
+----------+------------+
|  auto(l5)|        auto|
|manual(m5)|      manual|
|manual(m6)|      manual|
|  auto(av)|        auto|
+----------+------------+
only showing top 4 rows



In [14]:
mpg.select(regexp_extract("trans" , r"(\w+)", 1).alias("transmission")).show(4)

+------------+
|transmission|
+------------+
|        auto|
|      manual|
|      manual|
|        auto|
+------------+
only showing top 4 rows



### 3. Load the tips dataset as a spark dataframe.

In [75]:
#load tips in spark dataframe
tips = spark.createDataFrame(data("tips"))
tips.show(3)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
+----------+----+------+------+---+------+----+
only showing top 3 rows



a.What percentage of observations are smokers?

In [76]:
tips.count()

244

In [77]:
#group by smoker and count
tips.groupby("smoker").count().show()

+------+-----+
|smoker|count|
+------+-----+
|    No|  151|
|   Yes|   93|
+------+-----+



In [109]:
#now from groupby and count, make a new columne called percent using withColumn
tips.groupby("smoker").count().withColumn(
    "percent", round(col("count")/tips.count(),2)
    ).show()

+------+-----+-------+
|smoker|count|percent|
+------+-----+-------+
|    No|  151|   0.62|
|   Yes|   93|   0.38|
+------+-----+-------+



b. Create a column that contains the tip percentage

In [86]:
tips.select(
    tips.tip,
     tips.total_bill, 
     round((tips.tip/tips.total_bill * 100),2)
    .alias("tip_percent")
    ).show(4)

+----+----------+-----------+
| tip|total_bill|tip_percent|
+----+----------+-----------+
|1.01|     16.99|       5.94|
|1.66|     10.34|      16.05|
| 3.5|     21.01|      16.66|
|3.31|     23.68|      13.98|
+----+----------+-----------+
only showing top 4 rows



c. Calculate the average tip percentage for each combination of sex and smoker.

In [128]:
tips.groupBy("sex","smoker").agg(
    round(mean(tips.tip/tips.total_bill),3).alias("avg_tip_percent")).show()

+------+------+---------------+
|   sex|smoker|avg_tip_percent|
+------+------+---------------+
|  Male|    No|          0.161|
|  Male|   Yes|          0.153|
|Female|    No|          0.157|
|Female|   Yes|          0.182|
+------+------+---------------+



### 4. Use the seattle weather dataset referenced in the lesson to answer the questions below

In [130]:
from vega_datasets import data

In [131]:
weather = data.seattle_weather()
weather = spark.createDataFrame(weather)

In [132]:
weather.show(5)

+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|     8.9|     2.8| 6.1|   rain|
+-------------------+-------------+--------+--------+----+-------+
only showing top 5 rows



- Convert the temperatures to fahrenheit.


In [ ]:
# c to f: (0°C × 9/5) + 32 = 32°F

In [144]:
#spark rewrites over column if same column name is created
weather = weather.withColumn(
    "temp_max", (weather.temp_max * 9/5 + 32)
        ).withColumn("temp_min", (weather.temp_min * 9/5 + 32))
weather.show(6)
    

+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|   55.04|    41.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|   51.08|   37.04| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|   53.06|   44.96| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|   53.96|   42.08| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|   48.02|   37.04| 6.1|   rain|
|2012-01-06 00:00:00|          2.5|   39.92|   35.96| 2.2|   rain|
+-------------------+-------------+--------+--------+----+-------+
only showing top 6 rows



- Which month has the most rain, on average?
